In [2]:
from utils import *

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/accelerate/utils/imports.py:274: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


In [3]:
focus_logits, focus_cache = get_focus_logits_and_cache()

In [33]:
flipped_to_yours_logits : Float[Tensor, "layer pos row col"]= t.zeros((8, 59, 8, 8), device=device)
yours_to_yours_logits : Float[Tensor, "layer pos row col"]= t.zeros((8, 59, 8, 8), device=device)
is_yours_all = t.zeros((8, 59, 8, 8), device=device)
is_flipped_all = t.zeros((8, 59, 8, 8), device=device)
flipped_logits = t.zeros((8, 59, 8, 8), device=device)

for layer in range(1, 8):
    OV = model.OV.AB[layer]
    for pos in range(59):
        resid = focus_cache["resid_pre", layer][:, pos]
        flipped_probe = get_probe(layer-1, "flipped", "post")[0, :, :, :, :]
        yours_probe = get_probe(layer-1, "linear", "post")[0, :, :, :, :]
        flipped_results = einops.einsum(resid, flipped_probe, 'batch d_model, d_model row col options -> batch row col options')
        yours_results = einops.einsum(resid, yours_probe, 'batch d_model, d_model row col options -> batch row col options')
        is_flipped = flipped_results.argmax(dim=-1) == FLIPPED
        is_yours = yours_results.argmax(dim=-1) == YOURS
        flipped_probe = flipped_probe[:, :, :, FLIPPED]
        yours_probe = yours_probe[:, :, :, YOURS]
        flipped_probe_normalized = flipped_probe / t.norm(flipped_probe, dim=0)
        yours_probe_normalized = yours_probe / t.norm(yours_probe, dim=0)
        flipped_probe_normalized = einops.repeat(flipped_probe_normalized, 'd_model row col -> d_model batch row col', batch=resid.shape[0])
        yours_probe_normalized = einops.repeat(yours_probe_normalized, 'd_model row col -> d_model batch row col', batch=resid.shape[0])
        flipped_logit = einops.einsum(resid, flipped_probe_normalized, 'batch d_model, d_model batch row col ->batch row col')
        yours_logit = einops.einsum(resid, yours_probe_normalized, 'batch d_model, d_model batch row col ->batch row col')
        flipped_after_OV = einops.einsum(flipped_logit * flipped_probe_normalized, OV, 'd_model_in batch row col, head d_model_in d_model_out  -> head batch d_model_out row col')
        # flipped_after_OV = flipped_after_OV.sum(dim=0)
        yours_after_OV = einops.einsum(yours_logit * yours_probe_normalized, OV, 'd_model_in batch row col, head d_model_in d_model_out  -> head batch d_model_out row col')
        # yours_after_OV = yours_after_OV.sum(dim=0)
        yours_probe_new = get_probe(layer, "linear", "post")[0, :, :, :, YOURS]
        flipped_to_yours_logit = einops.einsum(flipped_after_OV, yours_probe_new, 'head batch d_model row col, d_model row col -> head batch row col')
        yours_to_yours_logit = einops.einsum(yours_after_OV, yours_probe_new, 'head batch d_model row col, d_model row col ->head batch row col')
        flipped_to_yours_logit *= is_flipped
        yours_to_yours_logit *= is_yours
        positive = yours_to_yours_logit.sum(dim=-1).sum(dim=-1).sum(dim=-1) > 0
        flipped_to_yours_logit = flipped_to_yours_logit[positive].sum(dim=0)
        yours_to_yours_logit = yours_to_yours_logit[positive].sum(dim=0)
        flipped_to_yours_logits[layer, pos] = (flipped_to_yours_logit).sum(dim=0)#  / is_flipped.sum(dim=0)
        yours_to_yours_logits[layer, pos] = (yours_to_yours_logit).sum(dim=0)#  / is_yours.sum(dim=0)
        is_yours_all[layer, pos] += is_yours.sum(dim=0)
        is_flipped_all[layer, pos] += is_flipped.sum(dim=0)

In [34]:
layer = 4
pos = 40

# replace nan with 0
flipped_to_yours_logits = t.nan_to_num(flipped_to_yours_logits, nan=0)
yours_to_yours_logits = t.nan_to_num(yours_to_yours_logits, nan=0)
flipped_to_yours_logit_plot = flipped_to_yours_logits[layer, :].sum(dim=0) / is_flipped_all[layer, :].sum(dim=0)
yours_to_yours_logit_plot = yours_to_yours_logits[layer, :].sum(dim=0) / is_yours_all[layer, :].sum(dim=0)
flipped_to_yours_logit_plot = t.nan_to_num(flipped_to_yours_logit_plot, nan=0)
yours_to_yours_logit_plot = t.nan_to_num(yours_to_yours_logit_plot, nan=0)
flipped_to_yours_logit_plot = flipped_to_yours_logit_plot[:].detach().cpu().numpy()
yours_to_yours_logit_plot = yours_to_yours_logit_plot[:].detach().cpu().numpy()
# flipped_to_yours_logit_plot = flipped_to_yours_logits[layer, pos].detach().cpu().numpy()
# yours_to_yours_logit_plot = yours_to_yours_logits[layer, pos].detach().cpu().numpy()
# Make plotly heatmaps of flipped_to_yours_logits and yours_to_yours_logits
'''fig = go.Figure()
fig.add_trace(go.Heatmap(z=flipped_to_yours_logit_plot, colorscale='Viridis'))
fig.update_layout(title=f"layer {layer} pos {pos} flipped_to_yours_logits")
fig.show()'''

# Make two heatmaps of flipped_to_yours_logits and yours_to_yours_logits as subplots
z_max = max(flipped_to_yours_logit_plot.max().item(), yours_to_yours_logit_plot.max().item())
z_min = min(flipped_to_yours_logit_plot.min().item(), yours_to_yours_logit_plot.min().item())
print(z_max, z_min)

fig = make_subplots(rows=1, cols=2, subplot_titles=("flipped_to_yours_logits", "yours_to_yours_logits"))
fig.add_trace(go.Heatmap(z=flipped_to_yours_logit_plot, colorscale='Viridis', zmin=z_min, zmax=z_max), row=1, col=1)
fig.add_trace(go.Heatmap(z=yours_to_yours_logit_plot, colorscale='Viridis', zmin=z_min, zmax=z_max), row=1, col=2)
fig.update_layout(title=f"layer {layer} pos {pos}")
fig.show()

0.9054391384124756 0.0


In [6]:
fig = go.Figure()
fig.add_trace(go.Heatmap(z=yours_to_yours_logit_plot, colorscale='Viridis'))
fig.update_layout(title=f"layer {layer} pos {pos} yours_to_yours_logits")
fig.show()

In [21]:
is_flipped_all = t.zeros((8, 59, 8, 8), device=device)
is_yours_all = t.zeros((8, 59, 8, 8), device=device)
is_mine_all = t.zeros((8, 59, 8, 8), device=device)
flipped_logits_all = t.zeros((8, 59, 8, 8), device=device)
yours_logits_all = t.zeros((8, 59, 8, 8), device=device)
mine_logits_all = t.zeros((8, 59, 8, 8), device=device)

for layer in range(0, 8):
    for pos in range(59):
        resid = focus_cache["resid_pre", layer][:, pos]
        flipped_probe = get_probe(layer, "flipped", "post")[0, :, :, :, :]
        linear_probe = get_probe(layer, "linear", "post")[0, :, :, :, :]
        flipped_results = einops.einsum(resid, flipped_probe, 'batch d_model, d_model row col options -> batch row col options')
        linear_results = einops.einsum(resid, linear_probe, 'batch d_model, d_model row col options -> batch row col options')
        is_flipped = flipped_results.argmax(dim=-1) == FLIPPED
        is_yours = linear_results.argmax(dim=-1) == YOURS
        is_mine = linear_results.argmax(dim=-1) == MINE

        flipped_probe = flipped_probe[:, :, :, FLIPPED]
        yours_probe = linear_probe[:, :, :, YOURS]
        mine_probe = linear_probe[:, :, :, MINE]
        flipped_probe_normalized = flipped_probe / t.norm(flipped_probe, dim=0)
        yours_probe_normalized = yours_probe / t.norm(yours_probe, dim=0)
        mine_probe_normalized = mine_probe / t.norm(mine_probe, dim=0)

        flipped_logits = einops.einsum(resid, flipped_probe_normalized, 'batch d_model, d_model row col -> batch row col') * is_flipped
        yours_logits = einops.einsum(resid, yours_probe_normalized, 'batch d_model, d_model row col -> batch row col') * is_yours
        mine_logits = einops.einsum(resid, mine_probe_normalized, 'batch d_model, d_model row col -> batch row col') * is_mine

        is_flipped_all[layer, pos] += is_flipped.sum(dim=0)
        is_yours_all[layer, pos] += is_yours.sum(dim=0)
        is_mine_all[layer, pos] += is_mine.sum(dim=0)

        flipped_logits_all[layer, pos] = (flipped_logits).sum(dim=0)
        yours_logits_all[layer, pos] = (yours_logits).sum(dim=0)
        mine_logits_all[layer, pos] = (mine_logits).sum(dim=0)

# Get the Average of the logits over all positions and layers

layer = 2
pos = 30

layer_start = 1
layer_end = 6
pos_start = 20
pos_end = 30

flipped_logits_all_new = flipped_logits_all[layer_start:layer_end, pos_start:pos_end].sum(dim=0).sum(dim=0) / is_flipped_all[layer_start:layer_end, pos_start:pos_end].sum(dim=0).sum(dim=0)
yours_logits_all_new = yours_logits_all[layer_start:layer_end, pos_start:pos_end].sum(dim=0).sum(dim=0) / is_yours_all[layer_start:layer_end, pos_start:pos_end].sum(dim=0).sum(dim=0)
# flipped_logits_all_new = flipped_logits_all[layer, pos] / is_flipped_all[layer, pos]
# yours_logits_all_new = yours_logits_all[layer, pos] / is_yours_all[layer, pos]
print(flipped_logits_all_new.round(decimals=2))
print(yours_logits_all_new.round(decimals=2))

tensor([[    nan,  0.2500,     nan,     nan,  0.9500,  1.2500, -0.2700,     nan],
        [ 1.1400,  1.1600,  0.9500,  0.8900,  0.9600,  0.9400,  1.0700,     nan],
        [ 0.7100,  1.0100,  0.8000,  0.8000,  0.8500,  0.8300,  1.0300,     nan],
        [ 0.0700,  1.0300,  0.7700,  0.8500,  0.7900,  0.8400,  0.9500,  0.7800],
        [ 0.9900,  0.9200,  0.9700,  0.8900,  0.9400,  0.8800,  0.9100,  0.9900],
        [ 0.4200,  1.2200,  0.8500,  0.8600,  0.9200,  0.8000,  1.1300,  0.9600],
        [ 1.4500,  0.8900,  0.9900,  0.8800,  0.9000,  0.9800,  1.1500,     nan],
        [    nan,  0.8700,  0.8000,  0.8600,  0.9700,  1.0600,  0.9800,     nan]],
       device='cuda:0')
tensor([[0.7600, 0.8300, 0.8900, 1.0200, 0.8600, 0.8500, 0.8300, 0.8600],
        [0.7800, 0.9300, 0.9100, 0.9700, 0.9500, 0.9200, 0.9600, 0.8000],
        [0.8600, 0.9200, 1.0300, 1.0900, 0.9900, 0.9300, 0.8500, 0.8100],
        [0.9100, 0.9800, 1.1600, 1.2200, 1.4400, 1.0300, 1.0900, 0.8500],
        [0.8000, 0.9700

In [ ]:
# The Logits for Flipped aren't actually Higher ... => This surprised me
# But empirically flipped seems to be more important, I haven't tested this out though
# Okay I don't have the time ... But maybe I can do this later (look at the logits when a tile has been flipped and write that)